# 🦆 Level 0: The Playground (In-Memory)

Welcome to the **DuckDB Laboratory**. This notebook is for rapid experimentation and testing relational logic without touching the disk.

### Why this matters:
- **Zero Setup**: No database files to manage.
- **Lazy Power**: Queries aren't run until you call `.show()` or `.df()`.
- **Variable Querying**: You can use Python variables as if they were SQL tables!

## 🏗️ 1. Setup & In-Memory Tables
We start by creating our e-commerce mini-world in RAM.

In [ ]:
import duckdb

# Create the 'users' table in memory
duckdb.sql("""
CREATE OR REPLACE TABLE users (
    user_id INTEGER,
    name VARCHAR,
    join_date DATE
);

INSERT INTO users VALUES
    (1, 'Alice', '2023-01-01'),
    (2, 'Bob', '2023-02-15'),
    (3, 'Charlie', '2023-03-10'),
    (4, 'Diana', '2023-04-20');
""")

# Create the 'orders' table in memory
duckdb.sql("""
CREATE OR REPLACE TABLE orders (
    order_id INTEGER,
    user_id INTEGER,
    amount DECIMAL(10, 2)
);

INSERT INTO orders VALUES
    (101, 1, 300.50),
    (102, 1, 250.00),
    (103, 2, 50.00),
    (104, 3, 600.00),
    (105, 4, 100.00),
    (106, 4, 150.00);
""")

print("✅ Success: 'users' and 'orders' created in-memory.")

## 👑 2. Case Study: VIP Hunter
We will join our tables to find customers who have spent more than $500.

In [ ]:
# The SQL way to hunt VIPs
vip_rel = duckdb.sql("""
    SELECT 
        u.name,
        SUM(o.amount) AS total_spend
    FROM users u
    JOIN orders o ON u.user_id = o.user_id
    GROUP BY u.name
    HAVING total_spend > 500
    ORDER BY total_spend DESC
""")

print("VIPs found!")
vip_rel.show()

## 🚄 3. Zero-Copy Bridge (DuckDB -> Arrow)
This is the 'Universal Language' of data. We move data to **Arrow** without any overhead (Zero-Copy).

In [ ]:
# Materialize the users table into an Arrow Table
arrow_table = duckdb.sql("SELECT * FROM users").to_arrow_table()

print("---------------------------------------------------")
print(f"Data Type: {type(arrow_table)}")
print("---------------------------------------------------")
print(f"Schema:\n{arrow_table.schema}")
print("---------------------------------------------------")
print(f"Arrow Table Contents:\n{arrow_table}")
print("---------------------------------------------------")

# Bonus: Convert back to Pandas for pretty Jupyter layout
arrow_table.to_pandas()

## 🚀 4. The Speed of Polars (Level 2 Preview)
Since Polars speaks Arrow natively, we can move our DuckDB data into a **Polars DataFrame** for high-speed analysis.

In [ ]:
# Materialize directly to Polars (Zero-Copy)
polars_df = duckdb.sql("SELECT * FROM users").pl()

print("---------------------------------------------------")
print(f"Data Type: {type(polars_df)}")
print("---------------------------------------------------")
print(f"Polars Schema:\n{polars_df.schema}")
print("---------------------------------------------------")
print(f"Polars DataFrame:\n{polars_df}")
print("---------------------------------------------------")

# Bonus: Convert back to Pandas for pretty Jupyter layout
polars_df.to_pandas()

## 💾 5. Persistent Export (Arrow -> Parquet)
Finally, we save our Arrow table to a physical Parquet file using the standard `pyarrow` library.

In [ ]:
import pyarrow.parquet as pq
import os

# Ensure the warehouse directory exists (relative to /notebooks/)
os.makedirs('../data/warehouse', exist_ok=True)

# Save to disk
pq.write_table(arrow_table, '../data/warehouse/arrow_export.parquet')

print("🎁 Filesystem Updated: data/warehouse/arrow_export.parquet is now ready for production!")

# 🧠 Knowledge Base: Common Questions

### Q1: What other options can we convert SQL tables to?
DuckDB is a "Universal Translator." You can convert results into almost any major format:
| Method | Target | Use Case |
| :--- | :--- | :--- |
| **`.df()`** | **Pandas** | The "Old Reliable." Best for small data. |
| **`.pl()`** | **Polars** | The "Speed Demon." Much faster than Pandas. |
| **`.fetchnumpy()`** | **NumPy** | Best for Machine Learning (tensors/arrays). |
| **`.fetchall()`** | **Python List** | Best for simple app logic. |
| **`.to_arrow_table()`** | **Arrow** | Best for interoperability. |

### Q2: `arrow_table.to_pandas()` — What is it and what else?
Arrow is **Memory-only**. It’s just a "Binary Block" of data. To interact with it, you convert it:
- **`.to_pandas()`**: Converts Arrow to a Pandas DataFrame (very fast).
- **`pl.from_arrow(arrow_table)`**: Move it into **Polars** (Zero-Copy king).
- **`arrow_table.to_pylist()`**: Convert to a standard Python list.
- **`arrow_table.to_batches()`**: Break it into small "chunks" for streaming.

### Q3: Why use Arrow to save to Parquet instead of just DuckDB?
Technically, DuckDB can save to Parquet directly: duckdb.sql("COPY table TO 'file.parquet'")

SO WHY USE ARROW?
1. **Engine Independence**: Arrow is a universal language. You can swap engines later without changing your save logic.
2. **Streaming**: Arrow is designed to "pipe" data between systems (e.g., Database to Web API).
3. **Metadata Control**: PyArrow gives you surgical control over Parquet settings (compression, etc.).
4. **Zero-Copy**: Arrow is a zero-copy format, meaning it can be read by multiple systems without copying the data.
🎓 The "Golden Rule" of this Stack:
Use DuckDB to query/crunch the data, but use Arrow to pass that data around. This is why we call it the "Zero-Copy" stack—Arrow is the bridge that keeps everything fast.

### Q4: Is Zero-Copy lost when using Polars (Level 2)?
**No!** Polars uses Apache Arrow as its internal memory format. When you call `.pl()`, DuckDB places data into an Arrow block, and Polars simply points its "eyes" at it. No copying happens.
```
graph LR
    A[DuckDB Engine] -- ".pl()" --> B(Arrow Memory Block)
    B -- "Zero Copy" --> C[Polars DataFrame]

```
DuckDB: Queries the data.
The Transfer: When you call .pl() in DuckDB, it doesn't "copy" the data into Polars. Instead, it places the data into an Arrow Memory Block.
Polars: Points its own "eyes" at that exact same memory block.

🧪 Proof you can see in your code:
```python
# This one line handles the whole "Zero-Copy" bridge automatically
polars_df = duckdb.sql("SELECT * FROM users").pl()
```

🧠 Why we use Polars instead of raw Arrow?
Raw Arrow Table: Great for storage and moving data, but very hard to code with. (Try doing a GROUP BY in raw Arrow—it's painful!)
Polars: Gives you a beautiful, easy API (like Pandas) to do complex data science, while still using the speed of Arrow underneath.
Summary:
You aren't losing the Zero-Copy logic in Level 2; you are harnessing it.

Arrow is the Track.
Polars is the High-Speed Train running on that track.

### Q5: What is the best practice for writing Parquet: `arrow_table` or `polars_df`?
| Method | Best For... | Why? |
| :--- | :--- | :--- |
| **`polars_df.write_parquet()`** | **Data Science & Logic** | Fast, multithreaded (Rust), and convenient. |
| **`pq.write_table(arrow_table)`** | **Infrastructure & Transport** | Maximum control over metadata and compatibility. |

1. Polars (polars_df.write_parquet())
Best For: When you are doing logic (filtering, joining, calculating).
The Difference: It uses a high-performance Rust-based writer. It is incredibly fast for local files because it is multithreaded by default.
Practice: If the data is already in a Polars DataFrame, always use df.write_parquet().
2. PyArrow (pq.write_table(arrow_table))
Best For: Pure infrastructure and "handing off" data.
The Difference: It is the "Industry Standard." It gives you surgical control. You can set specific Parquet versions, custom encryption, or compression levels (like Zstd level 22).
Practice: Use this when you are building a "bridge" between two systems (like DuckDB and a Cloud Bucket) and don't need to do any Python-based math on the data.

### Q6: What exactly is the "Arrow Bridge" (The Analogy)?
Think of your data flow like **Water**:

The arrow_table you created is the actual "Bridge" (the data itself), and pyarrow (pq) is the construction worker that uses that bridge to build a file.

Here is the breakdown of the "Bridge" in your code:

1. The Bridge (The Data Format)
Object: arrow_table
Role: This is the data sitting in your RAM. It is no longer "owned" by DuckDB. It is now in a universal format that Polars, Pandas, and Spark all understand perfectly.
Why we call it a bridge: Because it allows data to move between different software engines without being "re-written" or "re-formatted" (Zero-Copy).
2. The Worker (The Tool)
Object: import pyarrow.parquet as pq
Role: This is just the library (the "construction tool") that has the instructions for how to take an Arrow Table and save it as a Parquet file on your hard drive.
🎨 The Analogy:
Think of your data like Water:

DuckDB is a high-tech Pressure Tank. It's great for swirling the water around (SQL), but it's hard for other people to use that specific tank.
arrow_table is a Standard Bucket. You've poured the water out of the tank and into a bucket that everyone in the neighborhood recognizes.
pyarrow (pq) is a Funnel. It’s the tool you use to pour that bucket into a Glass Bottle (the Parquet file) so you can keep it in the fridge forever.
Summary:
DuckDB = The Engine (Querying).
arrow_table = The Bridge (The Memory-Format).